# Imports

In [1]:
import pandas as pd
import numpy as np
import psycopg2
import pandas as pd
import pandas.io.sql as psql
from sqlalchemy import create_engine, text
import yaml
from pathlib import Path
import os

project_dir = Path('__main__').resolve().parent.parent

# Conexão com ElephantSQL

In [2]:
#Preencher com os parametros do seu database do elephantsql
db_params = {
    'host': 'silly.db.elephantsql.com',
    'database': 'gpktqmyr',
    'user': 'gpktqmyr',
    'password': '7fteS2MDOlVYl1An-aQASuGjeADCX4hV'
}

In [3]:
#Criando conexão
conn = psycopg2.connect(
    host=db_params['host'],
    database=db_params['database'],
    user=db_params['user'],
    password=db_params['password']
)

# Criação de Tabela

In [4]:
# Create a cursor object
cur = conn.cursor()

# Set automatic commit to be true, so that each action is committed without having to call conn.committ() after each command
conn.set_session(autocommit=True)

In [ ]:
# Criar tabela
cur.execute('''
    CREATE TABLE phishing_data (
        id SERIAL PRIMARY KEY,
        url_length INTEGER,
        n_dots FLOAT,
        n_hypens FLOAT,
        n_underline FLOAT,
        n_slash FLOAT,
        n_questionmark FLOAT,
        n_equal FLOAT,
        n_at FLOAT,
        n_and FLOAT,
        n_exclamation FLOAT,
        n_space FLOAT,
        n_tilde FLOAT,
        n_comma FLOAT,
        n_plus FLOAT,
        n_asterisk FLOAT,
        n_hastag FLOAT,
        n_dollar FLOAT,
        n_percent FLOAT,
        n_redirection FLOAT,
        phishing FLOAT
    );

''')

In [ ]:
#Fechando conexão, boa prática
cur.close()
conn.close()

# Inserção de Dados

In [5]:
#Definindo a engine do banco de dados
engine = create_engine(f'postgresql://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}/{db_params["database"]}')

In [59]:
# Lendo os dados
df = pd.read_csv(os.path.join(project_dir, 'data', 'raw', 'web-page-phishingValidate.csv')).drop(columns= 'Unnamed: 0')
df.head()

,url_length,n_dots,n_hypens,n_underline,n_slash,n_questionmark,n_equal,n_at,n_and,n_exclamation,n_space,n_tilde,n_comma,n_plus,n_asterisk,n_hastag,n_dollar,n_percent,n_redirection,phishing
0,37,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,77,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,126,4.0,1.0,2.0,0.0,1.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,18,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,55,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [19]:
df.to_sql('phishing_data', engine, if_exists='replace', index=False)

77

# SQL

## Comandos

In [6]:
# Create a cursor object
cur = conn.cursor()

# Set automatic commit to be true, so that each action is committed without having to call conn.committ() after each command
conn.set_session(autocommit=True)

### SELECT, FROM, WHERE

In [7]:
query = '''
SELECT
  *
FROM Transactions LIMIT 10
'''

In [8]:
df_result = psql.read_sql(query, conn)
df_result

C:\Users\T-GAMER\AppData\Local\Temp\ipykernel_35720\1607457878.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = psql.read_sql(query, conn)


,Unnamed: 0.1,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:04:42,3.007469e+13,fraud_Rowe-Vandervort,grocery_net,44.54,Kelsey,Richards,F,889 Sarah Station Suite 624,...,37.9931,-100.9893,2691,Arboriculturist,1993-08-16,83ec1cc84142af6e2acf10c44949e720,1325376282,37.162705,-100.153370,0
1,1,2019-01-01 00:05:18,4.922711e+15,fraud_Herzog Ltd,misc_pos,4.27,Heather,Chase,F,6888 Hicks Stream Suite 954,...,40.3359,-79.6607,1472,Public affairs consultant,1941-03-07,fc28024ce480f8ef21a32d64c93a29f5,1325376318,40.351813,-79.958146,0
2,2,2019-01-01 00:06:01,2.720830e+15,"fraud_Schoen, Kuphal and Nitzsche",grocery_pos,198.39,Melissa,Aguilar,F,21326 Taylor Squares Suite 708,...,36.5220,-87.3490,151785,Pathologist,1974-03-28,3b9014ea8fb80bd65de0b1463b00b00e,1325376361,37.179198,-87.485381,0
3,3,None,4.642895e+12,fraud_Rutherford-Mertz,grocery_pos,24.74,Eddie,Mendez,M,1831 Faith View Suite 653,...,40.7491,-95.0380,7297,IT trainer,1990-07-13,d71c95ab6b7356dd74389d41df429c87,1325376383,40.275891,-96.011548,0
4,4,2019-01-01 00:11:14,2.348245e+15,"fraud_Konopelski, Schneider and Hartmann",food_dining,63.07,Justin,Gay,M,268 Hayes Rue Suite 811,...,42.1767,-79.9416,-999999,Event organiser,1946-02-02,8500f3d459047eac8443307b1e8296e5,1325376674,41.430274,-79.492553,0
5,5,2019-01-01 00:22:39,4.599735e+18,"fraud_Heller, Gutmann and Zieme",grocery_pos,82.80,Mary,Myers,F,39787 Pamela Fall Suite 639,...,36.3846,-89.4649,5577,Geochemist,1964-12-30,cb598ec00d349dc95e7b3c18de5ae800,1325377359,36.294205,-89.811282,0
6,6,2019-01-01 00:27:56,3.056121e+13,fraud_Harber Inc,gas_transport,82.22,Gina,Morrison,F,41851 Victor Drives Suite 219,...,40.5891,-75.4645,166081,"Scientist, research (maths)",1998-10-01,61bcb858dd5c8e2d0d46d02b4fa0bf8e,1325377676,40.586238,-75.802249,0
7,7,2019-01-01 00:29:11,4.789912e+15,"fraud_Mosciski, Gislason and Mertz",grocery_pos,221.81,Dorothy,Zuniga,F,77321 Ford Via,...,40.8095,-72.8229,3285,"Research officer, trade union",1988-04-27,a8394c3e555166ce2f90eee628bd9ade,1325377751,40.011959,-72.594808,0
8,8,2019-01-01 00:31:51,4.642255e+15,fraud_Ledner-Pfannerstill,gas_transport,102.13,Sabrina,Johnson,F,320 Nicholson Orchard,...,38.9999,-109.6150,-999999,"Surveyor, minerals",1987-04-23,47238da5b40d126c8abea40a857c7809,1325377911,39.807313,-109.348294,0
9,9,2019-01-01 00:32:15,3.007469e+13,fraud_Lockman Ltd,grocery_pos,212.75,Kelsey,Richards,F,889 Sarah Station Suite 624,...,37.9931,-100.9893,2691,Arboriculturist,1993-08-16,d9dcde500bed2cad48ee41e44c362596,1325377935,38.862183,-101.234087,0


In [16]:
query = '''
SELECT
  merchant,
  category,
  amt,
  last,
  gender,
  job,
  is_fraud
FROM Transactions LIMIT 10
'''

In [17]:
df_result = psql.read_sql(query, conn)
df_result

C:\Users\T-GAMER\AppData\Local\Temp\ipykernel_35720\1607457878.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = psql.read_sql(query, conn)


,merchant,category,amt,last,gender,job,is_fraud
0,fraud_Rowe-Vandervort,grocery_net,44.54,Richards,F,Arboriculturist,0
1,fraud_Herzog Ltd,misc_pos,4.27,Chase,F,Public affairs consultant,0
2,"fraud_Schoen, Kuphal and Nitzsche",grocery_pos,198.39,Aguilar,F,Pathologist,0
3,fraud_Rutherford-Mertz,grocery_pos,24.74,Mendez,M,IT trainer,0
4,"fraud_Konopelski, Schneider and Hartmann",food_dining,63.07,Gay,M,Event organiser,0
5,"fraud_Heller, Gutmann and Zieme",grocery_pos,82.80,Myers,F,Geochemist,0
6,fraud_Harber Inc,gas_transport,82.22,Morrison,F,"Scientist, research (maths)",0
7,"fraud_Mosciski, Gislason and Mertz",grocery_pos,221.81,Zuniga,F,"Research officer, trade union",0
8,fraud_Ledner-Pfannerstill,gas_transport,102.13,Johnson,F,"Surveyor, minerals",0
9,fraud_Lockman Ltd,grocery_pos,212.75,Richards,F,Arboriculturist,0


In [38]:
query = '''
SELECT
   merchant,
  category,
  amt,
  last,
  gender,
  job,
  is_fraud
FROM Transactions
WHERE is_fraud = 1 AND amt>=100
'''

In [39]:
df_result = psql.read_sql(query, conn)
df_result

C:\Users\T-GAMER\AppData\Local\Temp\ipykernel_35720\1607457878.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = psql.read_sql(query, conn)


,merchant,category,amt,last,gender,job,is_fraud
0,fraud_Rutherford-Mertz,grocery_pos,281.06,Murphy,M,Soil scientist,1
1,fraud_Goodwin-Nitzsche,grocery_pos,276.31,Hart,F,Horticultural consultant,1
2,fraud_Koepp-Parker,grocery_pos,275.73,Hart,F,Horticultural consultant,1
3,fraud_Medhurst PLC,shopping_net,844.80,Murphy,M,Soil scientist,1
4,fraud_Ruecker Group,misc_net,843.91,Murphy,M,Soil scientist,1
...,...,...,...,...,...,...,...
1135,fraud_Barrows PLC,shopping_pos,986.52,Sanders,M,Theatre director,1
1136,fraud_Jakubowski Inc,home,251.77,Smith,F,Magazine features editor,1
1137,fraud_Reichel Inc,entertainment,492.84,Smith,F,Magazine features editor,1
1138,fraud_Koelpin and Sons,misc_net,770.80,Thomas,F,Early years teacher,1


In [67]:
query = '''
SELECT
  url_length,
  n_dots,
  n_hypens,
  n_underline,
  n_slash,
  n_questionmark,
  phishing
FROM PHISHING_DATA
WHERE url_length >= 1000 AND n_slash > 0
'''

In [40]:
df_result = psql.read_sql(query, conn)
df_result

C:\Users\T-GAMER\AppData\Local\Temp\ipykernel_35720\1607457878.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = psql.read_sql(query, conn)


,merchant,category,amt,last,gender,job,is_fraud
0,fraud_Rutherford-Mertz,grocery_pos,281.06,Murphy,M,Soil scientist,1
1,fraud_Goodwin-Nitzsche,grocery_pos,276.31,Hart,F,Horticultural consultant,1
2,fraud_Koepp-Parker,grocery_pos,275.73,Hart,F,Horticultural consultant,1
3,fraud_Medhurst PLC,shopping_net,844.80,Murphy,M,Soil scientist,1
4,fraud_Ruecker Group,misc_net,843.91,Murphy,M,Soil scientist,1
...,...,...,...,...,...,...,...
1135,fraud_Turner and Sons,shopping_pos,875.12,Lane,M,Chief Technology Officer,1
1136,"fraud_Konopelski, Schneider and Hartmann",food_dining,128.49,Stewart,F,Health physicist,1
1137,fraud_Dooley-Thompson,misc_net,773.40,Lane,M,Chief Technology Officer,1
1138,fraud_Bashirian Group,shopping_net,992.20,Lane,M,Chief Technology Officer,1


In [71]:
query = '''
SELECT
  url_length,
  n_dots,
  n_hypens,
  n_underline,
  n_slash,
  n_questionmark,
  phishing
FROM PHISHING_DATA
WHERE n_questionmark > 3 OR n_slash > 2
LIMIT 10
'''

In [41]:
df_result = psql.read_sql(query, conn)
df_result

C:\Users\T-GAMER\AppData\Local\Temp\ipykernel_35720\1607457878.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = psql.read_sql(query, conn)


,merchant,category,amt,last,gender,job,is_fraud
0,fraud_Rutherford-Mertz,grocery_pos,281.06,Murphy,M,Soil scientist,1
1,fraud_Goodwin-Nitzsche,grocery_pos,276.31,Hart,F,Horticultural consultant,1
2,fraud_Koepp-Parker,grocery_pos,275.73,Hart,F,Horticultural consultant,1
3,fraud_Medhurst PLC,shopping_net,844.80,Murphy,M,Soil scientist,1
4,fraud_Ruecker Group,misc_net,843.91,Murphy,M,Soil scientist,1
...,...,...,...,...,...,...,...
1135,fraud_Turner and Sons,shopping_pos,875.12,Lane,M,Chief Technology Officer,1
1136,"fraud_Konopelski, Schneider and Hartmann",food_dining,128.49,Stewart,F,Health physicist,1
1137,fraud_Dooley-Thompson,misc_net,773.40,Lane,M,Chief Technology Officer,1
1138,fraud_Bashirian Group,shopping_net,992.20,Lane,M,Chief Technology Officer,1


### COUNT, AVG, SUM, MAX, GROUP BY, ORDER BY

In [185]:
query = '''
SELECT
  is_fraud,
  COUNT(is_fraud) as total
FROM Transactions
GROUP BY is_fraud
'''

In [186]:
df_result = psql.read_sql(query, conn)
df_result

C:\Users\T-GAMER\AppData\Local\Temp\ipykernel_35720\1607457878.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = psql.read_sql(query, conn)


,is_fraud,total
0,0,268827
1,1,1524


In [61]:
query = '''
SELECT
  category,
  COUNT(category) as type
FROM Transactions
WHERE is_fraud=1
GROUP BY category

'''

In [62]:
df_result = psql.read_sql(query, conn)
df_result

C:\Users\T-GAMER\AppData\Local\Temp\ipykernel_35720\1607457878.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = psql.read_sql(query, conn)


,category,type
0,entertainment,46
1,food_dining,33
2,gas_transport,134
3,grocery_net,27
4,grocery_pos,351
5,health_fitness,36
6,home,34
7,kids_pets,62
8,misc_net,175
9,misc_pos,55


In [178]:
query = '''
SELECT
  is_fraud,
  COUNT(is_fraud) as total,
  --AVG(url_length) as media_tamanho_url,
  SUM(amt) as amt
FROM Transactions
GROUP BY is_fraud
--ORDER BY AVG(url_length)
'''

In [179]:
df_result = psql.read_sql(query, conn)
df_result

C:\Users\T-GAMER\AppData\Local\Temp\ipykernel_35720\1607457878.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = psql.read_sql(query, conn)


,is_fraud,total,amt
0,0,268827,18120989.87
1,1,1524,776532.96


In [87]:
query = '''
SELECT
  phishing,
  COUNT(url_length) as qtd_linhas,
  AVG(url_length) as media_tamanho_url,
  SUM(url_length) as soma_tamanho_urls
FROM PHISHING_DATA
GROUP BY phishing
ORDER BY COUNT(url_length) DESC
'''

In [88]:
df_result = psql.read_sql(query, conn)
df_result

C:\Users\diego\AppData\Local\Temp\ipykernel_20224\1607457878.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = psql.read_sql(query, conn)


,phishing,qtd_linhas,media_tamanho_url,soma_tamanho_urls
0,0.0,62500,23.578016,1473626.0
1,1.0,35575,66.507266,2365996.0
2,NaN,2002,40.540959,81163.0


In [91]:
query = '''
SELECT
  phishing,
  n_hypens,
  COUNT(url_length) as qtd_linhas,
  AVG(url_length) as media_tamanho_urls,
  SUM(url_length) as soma_tamanho_urls,
  MAX(url_length) as tamanho_maximo_url
FROM PHISHING_DATA
WHERE n_hypens <= 2
GROUP BY phishing, n_hypens
ORDER BY COUNT(url_length) DESC
'''

In [92]:
df_result = psql.read_sql(query, conn)
df_result

C:\Users\diego\AppData\Local\Temp\ipykernel_20224\1607457878.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = psql.read_sql(query, conn)


,phishing,n_hypens,qtd_linhas,media_tamanho_urls,soma_tamanho_urls,tamanho_maximo_url
0,0.0,0.0,55552,21.292123,1182820.0,546
1,1.0,0.0,22884,55.649580,1273485.0,1962
2,1.0,1.0,8266,71.198040,588523.0,1307
3,0.0,1.0,4298,27.208004,116940.0,228
4,1.0,2.0,2240,92.696429,207640.0,4165
5,0.0,2.0,985,41.992893,41363.0,173


# 

### CASE WHEN

In [93]:
query = '''
SELECT
  CASE WHEN URL_LENGTH < 100 then 'PEQUENA'
       WHEN URL_LENGTH < 400 then 'MEDIA'
       WHEN URL_LENGTH < 600 then 'GRANDINHA'
      ELSE 'GRANDE'
  END AS TAMANHO_URL,
  URL_LENGTH,
  PHISHING
FROM PHISHING_DATA
LIMIT 10
'''

In [94]:
df_result = psql.read_sql(query, conn)
df_result

C:\Users\diego\AppData\Local\Temp\ipykernel_20224\1607457878.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = psql.read_sql(query, conn)


,tamanho_url,url_length,phishing
0,PEQUENA,37,0.0
1,PEQUENA,77,1.0
2,MEDIA,126,1.0
3,PEQUENA,18,0.0
4,PEQUENA,55,0.0
5,PEQUENA,32,1.0
6,PEQUENA,19,0.0
7,PEQUENA,81,1.0
8,PEQUENA,42,0.0
9,MEDIA,104,0.0


## Exemplos

In [24]:
# Create a cursor object
cur = conn.cursor()

# Set automatic commit to be true, so that each action is committed without having to call conn.committ() after each command
conn.set_session(autocommit=True)

In [95]:
# Verificando quantas linhas tenho na minha tabela, quantidade de links de
#phishing e o percentual total de links de phishing na minha tabela
query = '''
SELECT
  COUNT(*) AS QTD_LINHAS,
  SUM(PHISHING) AS QTD_LINKS_PHISHING,
  AVG(PHISHING) AS PERC_LINKS_PHISHING
FROM PHISHING_DATA
'''

In [96]:
df_result = psql.read_sql(query, conn)

C:\Users\diego\AppData\Local\Temp\ipykernel_20224\1564066013.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = psql.read_sql(query, conn)


In [97]:
df_result

,qtd_linhas,qtd_links_phishing,perc_links_phishing
0,100077,35575.0,0.362733


In [98]:
# Verificando quantidade de phishings e percentual baseado no tamanho da url
query = '''
SELECT
  URL_LENGTH,
  COUNT(*) AS QTD_LINHAS,
  SUM(PHISHING) AS QTD_PHISHING,
  AVG(PHISHING) AS PERC_PHISHING
FROM PHISHING_DATA
GROUP BY URL_LENGTH
'''

In [100]:
df_result = psql.read_sql(query, conn)
df_result

C:\Users\diego\AppData\Local\Temp\ipykernel_20224\1607457878.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = psql.read_sql(query, conn)


,url_length,qtd_linhas,qtd_phishing,perc_phishing
0,938,1,1.0,1.000000
1,184,6,6.0,1.000000
2,87,136,108.0,0.812030
3,477,2,2.0,1.000000
4,273,9,9.0,1.000000
...,...,...,...,...
485,536,1,1.0,1.000000
486,449,1,1.0,1.000000
487,64,500,412.0,0.839104
488,55,485,353.0,0.752665


In [101]:
# Agrupando o tamanho das url em pequenas, médias e grandes, e verificando as métricas
query = '''
SELECT
  CASE WHEN URL_LENGTH < 100 then 'PEQUENA'
       WHEN URL_LENGTH < 400 then 'MEDIA'
      ELSE 'GRANDE'
  END AS TAMANHO_URL,
  COUNT(*) AS QTD_LINHAS,
  SUM(PHISHING) AS QTD_PHISHING,
  AVG(PHISHING) AS PERC_PHISHING
FROM PHISHING_DATA
GROUP BY 1
ORDER BY COUNT(*)
'''

In [102]:
df_result = psql.read_sql(query, conn)
df_result

C:\Users\diego\AppData\Local\Temp\ipykernel_20224\1607457878.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = psql.read_sql(query, conn)


,tamanho_url,qtd_linhas,qtd_phishing,perc_phishing
0,GRANDE,149,141.0,0.959184
1,MEDIA,5878,5288.0,0.920613
2,PEQUENA,94050,30146.0,0.327020


## Pra vocês praticarem

Links de apoio: 
* https://learnsql.com/blog/sql-basics-cheat-sheet/
* 

## Dataset de Fraude

Utilize somente SQL para responder as perguntas abaixo


*   Qual o total de transações, total de transações fraudulentas e percentual de transações fraudulentas tem na sua tabela?
*   Qual os top 10 estabelecimentos com maior concentração de transações fraudulentas?
*   Em quais cidades estão concentrados 90% de todas as fraudes da sua tabela?
*   Qual número de conta tem mais transações fraudulentas? E qual número de conta tem a maior soma de valor de fraude?
*   Qual o total de transações, total de transações fraudulentas e percentual de transações fraudulentas por horários do dia?
*   Quais são os três meses com o maior número de transações?
*   Quais são os cinco tipos de categoria de transação que têm o maior número de transações fraudulentas?
*   Existem transações duplicadas na sua tabela? Se sim quantas?
*   Quais os top 10 clientes que sofreram fraude em mais meses diferentes?
*   Quais os empregos que concentram 50% de todas as fraudes da sua tabela?

In [221]:
query = '''
SELECT
  merchant,
  SUM(is_fraud) as total
FROM Transactions
GROUP BY 1
ORDER BY (total) DESC
LIMIT 10
'''

In [222]:
#2
df_result = psql.read_sql(query, conn)
df_result

C:\Users\T-GAMER\AppData\Local\Temp\ipykernel_35720\1607457878.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = psql.read_sql(query, conn)


,merchant,total
0,fraud_Kiehn-Emmerich,15.0
1,fraud_Kilback LLC,12.0
2,fraud_Kozey-Boehm,12.0
3,fraud_Heathcote LLC,12.0
4,fraud_Cole PLC,11.0
5,"fraud_Baumbach, Feeney and Morar",10.0
6,fraud_Koepp-Parker,10.0
7,"fraud_Greenholt, O'Hara and Balistreri",10.0
8,fraud_Gleason-Macejkovic,10.0
9,fraud_Goodwin-Nitzsche,10.0


In [225]:
query = '''
SELECT
  city,
  SUM(is_fraud) as total
FROM Transactions
GROUP BY 1
ORDER BY (total) DESC
LIMIT 10
'''

In [226]:
#3
df_result = psql.read_sql(query, conn)
df_result

C:\Users\T-GAMER\AppData\Local\Temp\ipykernel_35720\308241595.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = psql.read_sql(query, conn)


,city,total
0,None,52.0
1,Warren,21.0
2,Hampton,20.0
3,Hubbell,19.0
4,Hovland,18.0
5,Detroit,17.0
6,Hahira,16.0
7,Milford,16.0
8,Brownville,15.0
9,Cass,15.0


In [251]:
query = '''
SELECT
  city,
  SUM(is_fraud) as total,
  AVG(is_fraud) as pct
FROM Transactions
GROUP BY city
ORDER BY (pct) DESC
LIMIT 11
'''

In [252]:
#4
df_result = psql.read_sql(query, conn)
df_result

C:\Users\T-GAMER\AppData\Local\Temp\ipykernel_35720\3359443898.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = psql.read_sql(query, conn)


,city,total,pct
0,Houston,7.0,1.0
1,Greenport,10.0,1.0
2,Gaines,6.0,1.0
3,Clinton,12.0,1.0
4,Crouse,8.0,1.0
5,Coulee Dam,15.0,1.0
6,Claypool,7.0,1.0
7,Aurora,11.0,1.0
8,Chattanooga,7.0,1.0
9,Beacon,11.0,1.0


In [249]:
query = '''
SELECT
  city,
  is_fraud
  FROM TRANSACTIONS
  WHERE city = 'Greenport'
'''

## Dataset de Churn

Utilize somente SQL para responder as perguntas abaixo


*   Qual o total de clientes, total de clientes que realizaram churn e percentual churns tem na sua tabela?
*   Qual os top 2 paises com maior concentração de churn?
*   Em quais paises estão concentrados 90% de todos os clientes que tiveram churn?
*   Qual a idade média e o balanço médio dos clientes que tiveram churn?E dos que não tiveram?
*   Qual o total de clientes, total de churns e percentual de churn para as três faixas de renda a seguir [0-1000); [1000-3500); [3500-infinito)?
*   Qual o percentual de churn por faixas de idade?
*   Qual a mediana, p75, p90 e p99 da renda dos clientes que realizam churn?
*   O Score de Crédito é uma boa variavel para predizer a probabilidade do churn?
*   Acumulando a idade a partir de 18 anos, com quantos anos temos 50% de churn da sua tabela?

# Problemas do Coração
* Qual é a quantidade de pessoas em cada categoria de saúde geral?
* Quantas pessoas tiveram check-ups nos últimos 2 anos e quantas não tiveram?
* Qual é a média do índice de massa corporal para cada faixa etária?
* Quais são as diferentes combinações de histórico de tabagismo e consumo de álcool e qual a quantidade de pessoas e quantidade de pesssoas que tiveram um ataque do coração nessas combinações?
* Qual é o número de pessoas em cada faixa de consumo de frutas que também consomem vegetais verdes?
* Qual é o número total de pessoas e o total de pessoas que tiveram um ataque do coração que têm histórico de depressão e histórico de diabetes?
* Quais são as faixas etárias com o maior número de pessoas que relatam histórico de doença cardíaca?
* Qual é o consumo médio de batatas fritas para cada categoria de saúde geral?
* Quais são os valores únicos de índice de massa corporal entre as pessoas do sexo masculino?
* Existem duplicadas na sua tabela? Se sim quantas linhas?

# Acidentes de Transito
* Quantos acidentes ocorreram em cada dia da semana?
* Qual é o número total de acidentes em que estavam envolvidos veículos de transporte público?
* Qual é a contagem de acidentes em que o veículo tinha algum defeito e o acidente resultou em lesão grave?
* Qual é a distribuição de acidentes por tipo de veículo, agrupados por faixa etária do motorista?
* Qual é o número total de acidentes que ocorreram durante o dia e envolveram uma colisão entre veículos ?
* Qual é o número de acidentes por dia da semana, onde pelo menos uma pessoa envolvida no acidente tinha idade entre 31 e 50 anos?
* Qual é a média de vítimas em acidentes que ocorreram em áreas residenciais?
* Quais são os tipos de acidente mais comuns em áreas industriais ?
* Existem duplicadas na sua tabela? Se sim quantas linhas?
* Qual é a contagem de acidentes por sexo da vítima, onde a gravidade do acidente é 'Slight Injury' e a vítima estava atravessando a rua?

# League of Legends 10 minutes
* Quantos jogos resultaram em vitória para o time azul e quantos em derrota?
* Qual é o número médio de assistências por jogo, agrupado pelo resultado da partida?
* Qual é a diferença média de ouro entre os jogos em que o time azul venceu e os jogos em que o time azul perdeu?
* Quantos jogos tiveram a primeira eliminação feita pelo time azul e quantos pelo time vermelho?
* Qual é o total de torres destruídas pelo time azul em cada jogo, agrupado pelo resultado da partida?
* Qual é a média de experiência do time azul em jogos em que o time azul venceu, agrupada pela presença ou não do Heraldo?
* Qual é o número médio de abates pelo time azul em jogos em que o time azul perdeu, agrupado pelo número de dragões conquistados pelo time azul?
* Qual é o jogo com a maior diferença de ouro entre os times azul e vermelho?
* Qual é a média de torres destruídas pelo time azul em jogos em que o time azul venceu, agrupada pelo número de monstros de elite mortos pelo time azul?
* Existem duplicadas na sua tabela? Se sim quantas linhas?

# Musica
* Quantos respondentes usam o Spotify como seu serviço de streaming principal?
* Qual é a média de horas por dia que os respondentes passam ouvindo música enquanto trabalham, agrupados por seu serviço de streaming principal?
* Qual é a frequência média de ouvir música clássica entre os respondentes que relatam ansiedade, agrupados por seu serviço de streaming principal?
* Quais são os gêneros musicais favoritos mais comuns entre os respondentes que relatam insônia?
* Qual é a média de horas por dia que os respondentes passam ouvindo música enquanto trabalham, agrupados por sua faixa etária?
* Qual é a frequência média de ouvir música country entre os respondentes que relatam depressão e têm menos de 30 anos?
Quantos respondentes relatam ouvir música EDM "Muito frequentemente" e também têm mais de 60 anos?
* Qual é o gênero musical favorito mais comum entre os respondentes que relatam tocar um instrumento musical?
* Qual é a média de horas por dia que os respondentes passam ouvindo música enquanto trabalham, agrupados por se eles são ou não instrumentistas?
* Qual é a frequência média de ouvir música pop entre os respondentes que relatam ouvir música "Raramente" exploratória?

# Doenças e Sintomas
* Quantos tipos distintos de doenças estão presentes na tabela?
* Qual é a doença mais comum na tabela e quantas ocorrências dela existem?
* Quais são os sintomas mais comuns de todas as doenças listadas na tabela?
* Quantas vezes o sintoma "itching" aparece na tabela?
* Quantas doenças diferentes apresentam o sintoma "skin_rash"?
* Qual é a média de sintomas por doença na tabela?
* Quais são as três doenças com o maior número de sintomas distintos?
* Quantas doenças têm mais de 10 sintomas listados?
* Qual é a doença com o menor número de sintomas listados?
* Quantas doenças diferentes têm exatamente 10 sintomas listados?

# Credit Score
* Quantos clientes únicos estão presentes na tabela?
* Qual é o país com o maior número de clientes na tabela?
* Quantos clientes do sexo masculino existem na tabela?
* Qual é a média de idade dos clientes na tabela?
* Qual é o saldo médio dos clientes que são membros ativos?
* Quantos clientes estão inativos e têm um cartão de crédito?
* Qual é o número médio de produtos por país?
* Quantos clientes de cada país estão propensos a cancelar o serviço (churn)?
* Qual é o saldo total por país?
* Quais são os países onde a média de saldo é maior do que a média geral de saldo de todos os clientes?

# League of Legends Season
* Quantos campeões únicos estão presentes na tabela?
* Qual é o tier mais comum entre os campeões?
* Qual é a média de pontuação dos campeões por função?
* Qual é a média de KDA dos campeões por classe?
* Qual é o tier médio dos campeões por função?
* Qual é o número total de campeões por temporada (season)?
* Qual é o tier médio dos campeões por função apenas para as funções com mais de 50% de vitórias?
* Quantos campeões têm uma taxa de vitória superior a 50%?
* Qual é a função mais escolhida pelos jogadores?
* Qual é a função com a maior média de pontuação?

# FIFA
* Quantas partidas únicas foram disputadas?
* Qual é a média de gols marcados pelas equipes da casa por partida?
* Qual é a média de gols marcados pelas equipes visitantes por partida?
* Qual é a média de gols marcados por partida em torneios de qualificação para a Copa do Mundo?
* Qual é o continente com a média mais alta de gols marcados por partida?
* Qual foi o torneio com a maior média de gols marcados por partida?
* Quantas vezes a equipe da casa ganhou, perdeu ou empatou?
* Qual é a média de gols marcados pelas equipes da casa por partida apenas em amistosos?
* Qual é o país que teve o maior número de partidas neutras?
* Qual é a média de gols marcados pelas equipes visitantes por partida em partidas neutras?